# 決定木
- 

In [1]:
import pandas as pd
import numpy as np

### データ前処理

In [5]:
sample = pd.read_csv('../data/train.csv')
sample

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [42]:
sample.dtypes

# dtypeがobjectのカラム名を取得
object_columns = sample.select_dtypes(include=['object']).columns
object_columns


,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S
...,...,...,...,...,...
886,"Montvila, Rev. Juozas",male,211536,NaN,S
887,"Graham, Miss. Margaret Edith",female,112053,B42,S
888,"Johnston, Miss. Catherine Helen ""Carrie""",female,W./C. 6607,NaN,S
889,"Behr, Mr. Karl Howell",male,111369,C148,C


In [43]:
def drop_cols(data: pd.DataFrame, cols:list) -> pd.DataFrame:
    '''
    '''
    data = data.drop(cols, axis=1)
    return data

# usecase
test = drop_cols(sample, ['Name', 'Sex'])
test

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,26.0,0,0,111369,30.0000,C148,C


In [6]:
sample_test = pd.read_csv('../data/test.csv')

In [8]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(sample, test_size=0.3)

print(data_train.shape)
print(data_test.shape)

(623, 12)
(268, 12)


In [12]:
x_train = data_train[['Age', 'Fare']]
y_train = data_train['Survived']

x_test = data_test[['Age', 'Fare']]
y_test = data_test['Survived']

### モデル実装

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from matplotlib.colors import ListedColormap
import graphviz

In [13]:
clf_model = DecisionTreeClassifier(max_depth=3)
clf_model.fit(x_train, y_train) # 分析する値はnumericじゃないとだめ。エンコーディングのフェーズが必要。
print(clf_model.score(x_test, y_test))

0.6529850746268657


In [34]:
# class_names を文字列に変換しリスト化
# これは決定木を用いてデータをクラスごとに分けるとき、それらの内容をリスト形式で渡す必要がある。
class_names = list(map(str, y_train.unique()))


dot_data = export_graphviz(clf_model,
                           class_names=class_names,
                           feature_names=list(x_train.columns)) # feature_namesは、クラス分けに使用する判断基準となる変数をリストで渡す。
graph = graphviz.Source(dot_data)
graph.render('test', format='png')

'test.png'

In [24]:
list(x_train.columns)

['Age', 'Fare']

In [20]:
from sklearn.datasets import load_iris

iris = load_iris()
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [30]:
y_train.unique()

array([0, 1])

In [22]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [28]:
type(iris)

sklearn.utils._bunch.Bunch